# Loss run contextual and spatial  approach #
###  Main Alpha. Groping spatial search by topics, then extract spatial information related. Finally tokenize entities by a retrained nlp model ###



## Load transfer learning model ##

In [ ]:
"""
Loss runs extract target info by topics.

@author: Eduardo Santos, Alberto de Obeso, Romulo Troncos
date: Sep 20 
"""
import os 
import ast
import lossrun
from tensorflow.keras.utils import get_file
import gensim
import lossrun

In [ ]:
## Load re trained model
try:
    path = get_file('GoogleNews-vectors-negative300.bin.gz', 
        origin='https://s3.amazonaws.com/dl4j-distribution/'+'GoogleNews-vectors-negative300.bin.gz')
except:
    print('Error downloading')
    raise

model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:

#model.doesnt_match(string.split())

#model.most_similar('date')
#model.similarity('insurer','line')

In [132]:
"""
Study relation to topics 

"""

string = 'Line of  Policy Num retention Status Description Open By email  submitted  dated 3-Apr-17  Verna Bennett  wheelchair  onto the  was taken  Infirmary  fracture/dislocation Open Letter from  Patient  advising  malpractice  been filed  connection  occurred  fell from  being offloaded    Certain Underwriters at Lloyds - Certain'



# create a list for correlation score
contextual_relation_score = []
contextual_relation_word = []

# take every \w in text
elements_in_split = string.split()

relation_ratio = .1 # grammar correlation from -1 to 1
# filter 
for i in elements_in_split:

    try: 
        correlation = model.similarity('status', i)
                                        #topic  ,word 
        if (correlation > relation_ratio):
            contextual_relation_word.append(i)
            contextual_relation_score.append(correlation)
    except:
        #print(i + ' most be retrained or modify')
        contextual_relation_score.append(-1)
        pass

print(string)
print('.'*50)
print('Contextual realed words to insurer: ' + f'{contextual_relation_word}')

Line of  Policy Num retention Status Description Open By email  submitted  dated 3-Apr-17  Verna Bennett  wheelchair  onto the  was taken  Infirmary  fracture/dislocation Open Letter from  Patient  advising  malpractice  been filed  connection  occurred  fell from  being offloaded    Certain Underwriters at Lloyds - Certain
..................................................
Contextual realed words to insurer: ['Policy', 'retention', 'Status', 'the', 'was', 'been', 'filed', 'connection', 'being']


In [134]:
## 
model.most_similar('insurer')

[('insurers', 0.7716545462608337),
 ('Insurer', 0.7563024759292603),
 ('reinsurer', 0.7067841291427612),
 ('insurance', 0.7065000534057617),
 ('casualty_insurer', 0.6807657480239868),
 ('policyholders', 0.6805585622787476),
 ('Insurers', 0.6498398184776306),
 ('Insurance', 0.6475108861923218),
 ('policyholder', 0.6436665058135986),
 ('premiums', 0.615146279335022)]

## Create NLP entitites model


In [ ]:
import itertools # join list trhougth permutations
import numpy as np  # just in case math is needed
import random # shuffle index tool
import csv # cvs file format managemnent
import os

In [ ]:
## Generate data files
 
# create the days list
days = [str(i) for i in range(1,32)]
 
# create the monts list
monts = [str(i) for i in range(1,13)]
 
# crete the years list
years = [str(1900 + i) for i in range(0,150)]
 
# create the monts list in letters
monts_letter = ['Jan','Feb','Mar','Apr', 'MAY','JUN', 'JUL', 'AUG','sep','oct','nov','dec']
 
# join the monts width the days
days_monts = [day + '/' + mont for day in days for mont in monts] + [mont + '/' + day for day in days for mont in monts]
 
# joind days_monts to years
data_date_diag = [day_mont + '/' + year for day_mont in days_monts for year in years]# + [year + '/' + day for day in days for mont in monts]
 
# change the format diag for dash
data_date_dash = []
 
for i in range(len(data_date_diag)):
  data_date_dash.append(data_date_diag[i].replace('/','-'))
 
# create the days_monts whith letter
 
days_monts_letter = [day + '/' + mont for day in days for mont in monts_letter] + [mont + '/' + day for day in days for mont in monts_letter]
 
data_date_letter_diag = [day_mont + '/' + year for day_mont in days_monts_letter for year in years]
 
data_date_letter_dash = []
 
for i in range(len(data_date_diag)):
  data_date_letter_dash.append(data_date_letter_diag[i].replace('/','-'))
 
# join all formats
 
data_date = data_date_diag + data_date_dash + data_date_letter_diag + data_date_letter_dash

In [ ]:
# load external data from web scraping
 
DATA_PATH = 'data/data_base/'
 
_lists2list = lambda List: [item for sublist in List for item in sublist]
 
 
# 
#     DATA DISTRIBUTUION AS.
#      _______________    ___________________     ___________________________
#     |READ CVS FILES| => |TRANSFORM TO LIST|  => |PRE PROC LIST, CLEAN DATA|
#   
#      __________________________    __________
#     |MERGE NAMES AND LAS NAMES| => |SAVE DATA|
 
# read companies list 
with open(DATA_PATH + 'loss_runs _companies.csv', newline= '' ) as f:
  reader = csv.reader(f)
  data_companies = list(reader)
 
# read last names list
with open(DATA_PATH + 'loss_runs_last_name.csv', newline= '' ) as f:
  reader = csv.reader(f)
  data_lastnames = list(reader)
 
# read last names
with open(DATA_PATH + 'loss_runs_names.csv', newline= '' ) as f:
  reader = csv.reader(f)
  data_names = list(reader)
 
# list of list to list
 
# compnanies
data_companies_list = _lists2list(data_companies)
# last names
data_lastnames_list = _lists2list(data_lastnames)
# names
data_names_list = _lists2list(data_names)
 
 
# join names and last names lists
 
data_names = [name + ' ' + last for name in data_names_list for last in data_lastnames_list]
random.shuffle(data_names)

In [ ]:
# clean companies names
import string 

printable= set(string.printable)

#sent = data_companies_list[696]

for i in range(len(data_companies_list)):
   data_companies_list[i] = ''.join(filter(lambda x: x in printable, data_companies_list[i]))

data_companies = data_companies_list

In [ ]:
#Retrain model
# random insert abbreviation last names (i.e., Alexander P.)

random.shuffle(data_date)

TRAIN_DATA = []

for i in range(1000):
  #print(data_date[i])
  TRAIN_DATA.append((data_date[i], {"entities":[(0,len(data_date[i]),'DATE')]}))

for i in range(len(data_companies)):
  #print(data_date[i])
  TRAIN_DATA.append((data_companies[i], {"entities":[(0,len(data_companies[i]),'INSURER')]}))

for i in range(500):
  #print(data_date[i])
  TRAIN_DATA.append((data_names[i], {"entities":[(0,len(data_names[i]),'NAME')]}))


random.shuffle(TRAIN_DATA)
TRAIN_DATA

In [101]:
## Load existing spacy model for retraining
import spacy
nlp=spacy.load('en_core_web_sm')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

# disable pipe lines
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [102]:
import random
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

 {'ner': 3286.538015978205}
Losses {'ner': 3296.8200109170552}
Losses {'ner': 3308.7738002108213}
Losses {'ner': 3313.4043469892617}
Losses {'ner': 3325.872245238888}
Losses {'ner': 3336.033420489895}
Losses {'ner': 3341.511309372055}
Losses {'ner': 3354.3337333069917}
Losses {'ner': 3362.5549117492315}
Losses {'ner': 3371.705316560137}
Losses {'ner': 3385.0202404426213}
Losses {'ner': 3396.6756802962896}
Losses {'ner': 3408.6644342111226}
Losses {'ner': 3417.034900324213}
Losses {'ner': 3421.5455857680913}
Losses {'ner': 3431.987428979742}
Losses {'ner': 3437.470235125887}
Losses {'ner': 3447.1773476766225}
Losses {'ner': 3451.3503992127057}
Losses {'ner': 3469.349338726866}
Losses {'ner': 3483.0752720402356}
Losses {'ner': 3486.489025401153}
Losses {'ner': 3494.794946836509}
Losses {'ner': 3503.3968552818674}
Losses {'ner': 3515.6513594856638}
Losses {'ner': 3522.501415714823}
Losses {'ner': 3527.997783825003}
Losses {'ner': 3542.4281642289984}
Losses {'ner': 3561.3186462374556}
Loss

In [ ]:
from spacy import displacy

text = 'Pedro Ramirex'

doc = nlp(text)

for ent in doc.ents:
    print(ent, ent.label_)
displacy.serve(doc, style='ent')

In [114]:
string

"Carrier:  Line of  Policy Num retention Status Description Open By email  submitted  dated 3-Apr-17  Verna Bennett  wheelchair  onto the  was taken  Infirmary  fracture/dislocation Open Letter from  Patient's  advising  malpractice  been filed  connect  occurred  fell from  being offloaded    Certain Underwriters at Lloyds - Certain    Loss Report prepared for A-MMED Ambulance Inc., DBA A-Med Carrier:"